# Subchallenge Challenge 2 Training notebook

* Use only solicited for training and validation
* Use BCE for loss function
* Scaled images to 0-1
* Used frequencies from approx 50 Hz to 15000Hz
* inference on test set is per patient, not per cough.  provide confidence score

* here we use standard spectrum bands similar to paper below.

https://www.researchgate.net/publication/323786906_Detection_of_tuberculosis_by_automatic_cough_sound_analysis

* Use CNN and a dense network for tabular data




In [1]:
import os
from pathlib import Path
import pandas as pd
import torch
import torchaudio
import torchvision
import numpy as np
from torchvision import models
import torchaudio
import matplotlib.pyplot as plt
from tqdm import tqdm
import timm
import gc
import librosa

ModuleNotFoundError: No module named 'torch'

In [ ]:
data_dir = Path('/media/SSD/CODA-TB/data')
solicited = data_dir/'solicited'
TRAIN_FOLDER = solicited

In [ ]:
MODEL_FOLDER = Path('/media/SSD/CODA-TB/models')

os.makedirs(MODEL_FOLDER, exist_ok=True)

## Load the metadata

In [ ]:
challenge1_df = pd.read_csv(data_dir/'metadata'/"challenge1_metadata.csv",index_col=[0])

In [ ]:
challenge2_df = pd.read_csv(data_dir/'metadata'/"challenge2_metadata.csv",index_col=[0])

In [ ]:
challenge2_df.columns

### Find data with both file and clinical data

In [ ]:
tab_cols = ['sex', 'age', 'height', 'weight', 'reported_cough_dur',
       'tb_prior', 'tb_prior_Pul', 'tb_prior_Extrapul', 'tb_prior_Unknown',
       'hemoptysis', 'heart_rate', 'temperature', 'weight_loss', 'smoke_lweek',
       'fever', 'night_sweats']

cat_cols = ['sex','tb_prior', 'tb_prior_Pul', 'tb_prior_Extrapul', 'tb_prior_Unknown',
            'hemoptysis', 'weight_loss', 'smoke_lweek','fever', 'night_sweats']

In [ ]:
data_df = pd.merge(challenge1_df, challenge2_df[['participant']+tab_cols], on = 'participant', how = 'inner')

### Encode the categorical data 

In [ ]:
def convert(s):
    lu = {'Male': -1, 'Female':1, 'Yes':1, 'No':-1, 'Not sure':0}
    s = s.apply(lambda x:lu[x])
    return(s)

In [ ]:
challenge2_df = data_df.apply(lambda x: convert(x) if x.name in cat_cols else x)

In [ ]:
challenge2_df

## Datasets

In [ ]:
#Librosa version
class Dataset_from_df(torch.utils.data.Dataset):
    def __init__(self, file_df, path, crop_length=22050):
        self.file_df = file_df
        self.path = path
        self.crop_length = crop_length
        
        self.tab_cols = ['sex', 'age', 'height', 'weight', 'reported_cough_dur',
                         'tb_prior', 'tb_prior_Pul', 'tb_prior_Extrapul', 'tb_prior_Unknown',
                         'hemoptysis', 'heart_rate', 'temperature', 'weight_loss', 'smoke_lweek',
                         'fever', 'night_sweats']
        

    def __len__(self):
        return len(self.file_df)

    def __getitem__(self, index):
        def _randint(a, b):
            return torch.randint(a, b, (1, )).item()
        
        #meta = torchaudio.info(self.path + "/" + df.iloc[index]['subpath'])
        target = self.file_df.iloc[index]['tb_status']*1.0
        length = self.file_df.iloc[index]['samples']
        
        # get tabular data
        tabular = self.file_df.iloc[index][self.tab_cols].to_numpy(dtype='float32')
        
        p = self.path/self.file_df.iloc[index]['filename']
        #p = self.file_df.iloc[index]['path']
        orig_id = self.file_df.iloc[index]['orig_id']

        if length == self.crop_length:
            x, sr = librosa.load(str(p),sr=None)
            
        elif length < self.crop_length:
            s, sr = librosa.load(str(p),sr=None)

            zeros_front = (_randint(0, self.crop_length - length))
            pad_front = np.zeros(( zeros_front))
            pad_back = np.zeros(( self.crop_length - length - zeros_front))
            waveform = np.concatenate((pad_front, s, pad_back), 0)
            x = waveform    
            
            
            
        else:
            crop_start = _randint(0, int(length - self.crop_length))
            x, sr = librosa.load(str(p),sr=None)
            x = x[crop_start:self.crop_length+crop_start]
            
        # transform sound sample to spectrum
        x = np.abs(librosa.stft(x,n_fft = 1024,hop_length=64,window ='hanning'))
        x = librosa.amplitude_to_db(x).astype(np.float32)
        x = (x-np.min(x))/(np.max(x)-np.min(x))
        x = np.flip(x[1:349,:],0).copy() # approx 50Hz to 15000Hz
  
        
        x=torch.from_numpy(x).unsqueeze(0)
        #print(x.shape)
        
        

        return {'x':x, 'sr':sr, 'tabular':tabular,'target':torch.FloatTensor([target, 1.0-target]),'orig_id':orig_id}
    

In [ ]:
def prepare_datasets(df, fold):
    train_df = df.query("fold!=@fold").reset_index(drop=True)
    valid_df = df.query("fold==@fold").reset_index(drop=True)
    
    #balance the data set
    df_counts = pd.DataFrame(train_df.groupby(['tb_status']).size().reset_index()).rename(columns={0:"counts"})
    df_counts['weights'] = df_counts['counts'].max() / df_counts['counts']
    df_balanced = pd.merge(train_df, df_counts[['tb_status','weights']], on='tb_status')
    sampler = torch.utils.data.WeightedRandomSampler(df_balanced['weights'].values, len(df_balanced))

    train_dataset = Dataset_from_df(train_df,TRAIN_FOLDER)
    valid_dataset = Dataset_from_df(valid_df,TRAIN_FOLDER)
    
    return train_dataset, valid_dataset, sampler

## Define the network

In [ ]:
class ChannelExpand(torch.nn.Module):
    def __init__(self, channels):
        super(ChannelExpand, self).__init__()
        self.channels = channels
    def forward(self,x):
        #print(x.shape)
        x = x.expand(-1,3,*x.shape[2:])
        
        return x

In [ ]:
model = timm.create_model('resnet34', pretrained=True,drop_rate = .50, num_classes=2)

In [ ]:
class MultiModel(torch.nn.Module):
    def __init__(self):
        super(MultiModel, self).__init__()
        self.cnn = timm.create_model('resnet34', pretrained=True,drop_rate = .50, num_classes=2)
        self.cnn.fc = torch.nn.Linear(
            self.cnn.fc.in_features, 20)  #modify last layer to 20 features (from 512)
        
        self.fc1 = torch.nn.Linear(20 + 16, 60) # dense layer 
        self.fc2 = torch.nn.Linear(60, 2) # output layer
        
        self.to3_channel = ChannelExpand(3)
        
    def forward(self, image, data):
        image = self.to3_channel(image)
        x1 = self.cnn(image)
        x2 = data
        
        x = torch.cat((x1, x2), dim=1)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
om_descr = 'timm-resnet34-c2'

def build_model():
    # build a new model
    model = MultiModel()

    
    return model

## Training and Inference Functions

In [ ]:
def trainfp32(epoch, train_dataloader, optimizer, loss_fn, lr_scheduler,
              metric_fn, model):



    model.train()

    training_loss = 0
    metric = 0

    m = torch.nn.Softmax(dim=1)
    pbar = tqdm(total=len(train_dataloader))

    for batch in train_dataloader:

        optimizer.zero_grad()


        x = batch['x']
        d = batch['tabular']
        sr = batch['sr']

        y = batch['target']

        x = x.cuda(non_blocking=True)
        d = d.cuda(non_blocking=True)
        y = y.cuda(non_blocking=True)

        output = model(x,d)
        
        loss = loss_fn(m(output), y)

        loss.backward()

        optimizer.step()
        

        lr_scheduler.step()

        training_loss += loss.data.item() * x.size(0)

        metric += metric_fn(y, output) * x.size(0)

        pbar.update(1)
    pbar.close()
    training_loss /= len(train_dataloader.dataset)
    metric /= len(train_dataloader.dataset)
    return training_loss, metric

In [ ]:
def inferfp32(valid_dataloader, model):
    act = np.zeros(len(validation_dataloader.dataset))
    pred = np.zeros(len(validation_dataloader.dataset))
    orig_id = np.zeros(len(validation_dataloader.dataset))

    m = torch.nn.Softmax(dim=1)
    
    model.eval()
    pbar = tqdm(total=len(valid_dataloader))

    st = 0
    for batch in valid_dataloader:
        x = batch['x']
        d = batch['tabular']
        sr = batch['sr']
        y = batch['target']
        x = x.cuda(non_blocking=True)
        d = d.cuda(non_blocking=True)
        
        output = model(x,d)
        
        
        en = st + y.shape[0]
        act[st:en] = y[:,0].numpy()
        pred[st:en] = m(output)[:,0].cpu().detach().numpy()
        orig_id[st:en] = batch['orig_id']
        st = en
        
        pbar.update(1)
    pbar.close()
    
    
    return act, pred, orig_id

In [ ]:
def plot_metrics(loss, metric):
    fig, ax = plt.subplots(2,1,figsize=(15,15))
    loss_line, = ax[0].plot(loss, label='loss')
    metric_line, = ax[1].plot(metric, label='metric')
    ax[0].legend(handles=[loss_line])
    ax[1].legend(handles=[metric_line])
    ax[0].set_title( "loss")
    ax[1].set_title( "metric")
    plt.show()

## Metrics

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import auc

def p_auroc(act,pred,fpr_thresh=1.0,tpr_thresh=0.0):
    fpr, tpr, _ = roc_curve(act, pred)
    #roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
    print('auroc=',roc_auc_score(act,pred))
    i = fpr <= fpr_thresh
    j = tpr >= tpr_thresh
    
    if len(fpr[i&j]) > 1 :
        pauc_approx = auc(fpr[i&j], tpr[i&j]-tpr_thresh)

        max_ij = np.argmax(fpr[i&j])
        pauc_extra = (fpr_thresh-fpr[i&j][max_ij]) * (tpr[i&j][max_ij]-tpr_thresh)
        pauc_better = pauc_approx + pauc_extra
    else:
        pauc_better = 0

    return pauc_better 
    

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, average_precision_score


def f1_torch(output, target):
    y_pred = output.argmax(axis=1)
    y_true = target.argmax(axis=1)
    np_true = y_true.to("cpu").to(torch.int).numpy()
    np_pred = y_pred.to("cpu").to(torch.int).numpy()
    return f1_score(np_true, np_pred, average="micro", zero_division=0)


### Training Parameters

In [ ]:
init_lr = .0005
num_epochs=30

loss_fn = torch.nn.CrossEntropyLoss()
num_folds = 5
load_weights = False
training_phase = 1


## Training Loop

In [ ]:
for fold in range(0,num_folds):
    
    model = build_model()
    model.cuda()
    
    if load_weights:
        model.load_state_dict(torch.load(MODEL_FOLDER / f"{fold}_dict_{om_descr}_{training_phase}.pth"))
    
    train_dataset, valid_dataset, sampler = prepare_datasets(challenge2_df,fold)
    
    train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                   batch_size=32,
                                                   num_workers=10,
                                                   pin_memory=True,
                                                   shuffle=False,
                                                   sampler=sampler)

    validation_dataloader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                                       batch_size=32,
                                                       num_workers=10,
                                                       pin_memory=True,
                                                       shuffle=False)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)
    lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=init_lr,
                                                       steps_per_epoch=len(train_dataloader),epochs=num_epochs)
    
    epoch_loss_values = list()
    metric_values = list()

    best_metric = -1
    print("-" * 10)
    print(f"fold {fold + 1}/{num_folds}")    
    
    for epoch in range(1, num_epochs + 1):

        lr_val = optimizer.param_groups[0]["lr"]
        print(f"Fold:{fold} epoch {epoch}/{num_epochs}  LR={lr_val}")

        training_loss, training_metric = trainfp32(epoch,
                                                   train_dataloader,
                                                   optimizer,
                                                   loss_fn,
                                                   lr_scheduler,
                                                   metric_fn=f1_torch,
                                                   model=model)

        print('Training-- Loss: {:.4f}, Metric: {:.3f}'.format(
            training_loss, training_metric),
              end=',')
        act, pred, _= inferfp32(validation_dataloader, model)
        
        p_auroc_metric = p_auroc(act,pred,fpr_thresh=0.4,tpr_thresh=0.8)
        auroc_metric = roc_auc_score(act,pred)
        f1_metric = f1_score(act,pred>0.5, average="micro", zero_division=0)
        
        validation_metric = auroc_metric


        print(f'Validation--  Metrics: auroc={auroc_metric:.3f}, p_auroc={p_auroc_metric:.5f}, f1={f1_metric:.3f}',end='\n')

        epoch_loss_values.append(training_loss)
        metric_values.append(validation_metric)
        
        if validation_metric > best_metric:
            best_metric = validation_metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), MODEL_FOLDER / f"{fold}_best_dict_{om_descr}_{training_phase}.pth")
            print("saved new best metric model")        
 
    plot_metrics(epoch_loss_values, metric_values)
    torch.save(model.state_dict(), MODEL_FOLDER / f"{fold}_dict_{om_descr}_{training_phase}.pth")

    model.cpu()
    del model, optimizer, train_dataloader, validation_dataloader, lr_scheduler
    gc.collect()
    torch.cuda.empty_cache()


## Inference

### Best Models

In [ ]:
load_weights = True
training_phase = 1

In [ ]:
act = np.zeros(len(challenge1_df))
pred = np.zeros(len(challenge1_df))
orig_id = np.zeros(len(challenge1_df))
num_folds = 5
st = 0

for fold in range(0,num_folds):
    
    model = build_model()
    model.cuda()
    

    model.load_state_dict(torch.load(MODEL_FOLDER / f"{fold}_best_dict_{om_descr}_{training_phase}.pth"))
    
    train_dataset, valid_dataset, _ = prepare_datasets(challenge2_df,fold)
    

    validation_dataloader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                                       batch_size=16,
                                                       num_workers=12,
                                                       pin_memory=True,
                                                       shuffle=False)

    model.eval()
    model.cpu()
    
    #get real data as dummy for ONNX
    iterator_loader = iter(validation_dataloader)
    batch = next(iterator_loader)
    dummy_input1 = batch['x']
    dummy_input2 = batch['tabular']
    dummy_input = (dummy_input1,dummy_input2 )
    
    filename = f"{fold}_best_dict_{om_descr}_{training_phase}.onnx"
    file_path = MODEL_FOLDER / filename
    
    
    
 
    # save the network
    torch.onnx.export(model,         # model being run 
         dummy_input,       # model input (or a tuple for multiple inputs) 
         file_path,       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['modelInput1','modelInput2'],   # the model's input names 
         output_names = ['modelOutput'], # the model's output names 
         dynamic_axes={'modelInput1' : {0 : 'batch_size'},'modelInput2' : {0 : 'batch_size'},    # variable length axes 
                                'modelOutput' : {0 : 'batch_size'}}) 
    
   
    
    model.cuda()
    
    #Convert_ONNX(model,dummy_input,file_path)
    
    act_fold,pred_fold,orig_id_fold = inferfp32(validation_dataloader, model)

    en = st + act_fold.shape[0]
    act[st:en] = act_fold
    pred[st:en] = pred_fold
    orig_id[st:en] = orig_id_fold
    st = en


    model.cpu()

    
    
    

In [ ]:
inference_df = pd.DataFrame({'orig_id':orig_id, 'act':act,'pred':pred})

In [ ]:
inference_df[inference_df['act'] == 0]

In [ ]:
inference_df = pd.merge(inference_df,challenge2_df, left_on='orig_id', right_index=True)
inference_df.drop(['orig_id_x','orig_id_y'],axis=1, inplace=True)

In [ ]:
res_df = inference_df.groupby('participant')[['pred','act']].agg(np.mean)

In [ ]:
res_df.to_csv('inference_ch2_df.csv')

In [ ]:
# Use the mean values as a quick way to get predicted and actual by participant
pred = res_df['pred'].values
act = res_df['act'].values

In [ ]:
from auroc_confidence import auc_metric

In [ ]:
_, auroc_bs_score, auroc_bs_score_ci, _, _ = auc_metric.AUC_CI_Bootstrap(act, pred, CI_index=0.95)

In [ ]:
pauroc_metric = p_auroc(act,pred,fpr_thresh=0.4,tpr_thresh=0.8)

In [ ]:
roc_auc_score(act,pred)

In [ ]:
f1 = f1_score(act,(pred>0.5), average="micro", zero_division=0)

In [ ]:
a = auroc_bs_score
b = auroc_bs_score_ci[0]
c = auroc_bs_score_ci[1]

print(f"Validation AUROC:{a:.3f} , ci=[{b:.3f}, {c:.3f}]")
print(f"Validation P-AUROC:{pauroc_metric:.3f}")
print(f"F1 Validation Score:{f1:.3f}")